In [2]:
import torch 
import torch.nn as nn
#import torchvision
#import torchvision.transforms as transforms
from dataset import Dataset
from ConvNet import ConvNet
import numpy as np
import lib
from sklearn import metrics as sk_metrics
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [3]:
# Datasets

partition = np.load('ID.npy')# IDs
labels = np.load('labels.npy')# Labels
partition = partition.item()
labels = labels.item()

# Drop irrelevant classes and reorder
# anger, anticipation, disgust fear, joy, sadness
#  0,     #1,         ,   2     , 7,   8, 15
#for i in range(labels.shape[0]): # pick an irrelevant column as dummy column
#    if labels[i, 1] == 0. & :
#        labels[i,2] = 1
#    else: labels[i,2] = 0
#labels = labels[:,[0,1,2,7,8,15]]  # column 2 is useless, set to 0 in the next line
#labels[:,2] = 0. 

# Hyper parameters
num_epochs = 5
num_classes = 6
learning_rate = 0.00001
params = {'batch_size': 32,
          'shuffle': True,
          'num_workers': 6}

# Generators
training_set = Dataset(partition['train'], labels)
training_generator = torch.utils.data.DataLoader(training_set, **params)

test_set = Dataset(partition['test'], labels)
test_generator = torch.utils.data.DataLoader(test_set, **params)


def init_weights(m):
    if type(m) == nn.Conv2d:
        nn.init.xavier_uniform_(m.weight)
        #m.bias.data.fill_(0.01)



In [6]:
model = ConvNet(num_classes).to(device)
model.apply(init_weights)
model.train()
# Loss and optimizer
#criterion = nn.MultiLabelSoftMarginLoss()
#criterion = nn.CrossEntropyLoss()
criterion = nn.MultiLabelMarginLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(training_generator)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(training_generator):
        #print(images)
        
        images = images.to(device).float()
        labels = labels.to(device).long()
        #print (images.shape)
        # Forward pass
        outputs = model(images)
        #print (labels)
        #print(outputs)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if ( i+1 % 5) == 0:
            print(labels)
            print(outputs)
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

torch.save(model.state_dict(), 'model/model.pth')

Epoch [1/5], Step [1/15], Loss: 4.4399
Epoch [1/5], Step [2/15], Loss: 2.7939
Epoch [1/5], Step [3/15], Loss: 1.4972
Epoch [1/5], Step [4/15], Loss: 1.5961
Epoch [1/5], Step [5/15], Loss: 0.7563
Epoch [1/5], Step [6/15], Loss: 0.9834
Epoch [1/5], Step [7/15], Loss: 1.0879


Process Process-19:
Process Process-23:
Process Process-21:
Process Process-22:
Process Process-24:
Process Process-20:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/ericchiu/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/ericchiu/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/ericchiu/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/Users/ericchiu/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/ericchiu/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/ericchiu/anaconda3/lib/python3.6/multiprocessing/pro

KeyboardInterrupt: 

In [7]:
outputs

tensor([[9.9704e-01, 1.0338e-03, 4.7375e-04, 4.4362e-04, 3.8701e-04, 6.2061e-04],
        [9.7862e-01, 6.2513e-03, 3.9451e-03, 3.5345e-03, 3.3957e-03, 4.2516e-03],
        [9.9570e-01, 1.2359e-03, 7.9095e-04, 8.0001e-04, 6.7290e-04, 8.0082e-04],
        [1.0000e+00, 1.5952e-16, 2.5442e-18, 8.7632e-19, 5.7560e-19, 5.6389e-18],
        [9.9999e-01, 3.1297e-06, 5.8669e-07, 4.4556e-07, 4.0640e-07, 7.9358e-07],
        [1.0000e+00, 3.3018e-12, 4.1482e-13, 2.5048e-13, 1.6111e-13, 4.0611e-13],
        [8.3723e-01, 3.7811e-02, 3.1745e-02, 3.1675e-02, 3.0927e-02, 3.0615e-02],
        [1.0000e+00, 1.2259e-15, 4.9583e-17, 2.4830e-17, 1.2687e-17, 4.1176e-17],
        [1.0000e+00, 4.4981e-11, 5.7636e-12, 5.4351e-12, 3.6063e-12, 8.5778e-12],
        [9.8437e-01, 4.1615e-03, 3.0014e-03, 2.7684e-03, 2.5279e-03, 3.1749e-03],
        [9.9995e-01, 2.8222e-05, 6.6791e-06, 5.6034e-06, 4.4168e-06, 8.6842e-06],
        [9.9841e-01, 4.6972e-04, 3.1282e-04, 2.3958e-04, 2.4038e-04, 3.2647e-04],
        [1.0000e

In [12]:
# Test the model
  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_generator:
        images = images.to(device).float()
        labels = labels.to(device).float()
        outputs = model(images)
        #_, predicted = torch.max(outputs.data, 1)
        predicted = outputs > 0.1666
        print ('predicted:', predicted)
        print ('target: ', labels)
        average_precision = sk_metrics.average_precision_score(labels, predicted)
        #total += labels.size(0)
        #correct += (predicted == labels).sum().item()

    print('Test Average Precision: {} %'.format(average_precision))

# Save the model checkpoint
#torch.save(model.state_dict(), 'model.ckpt')

predicted: tensor([[1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0]], dtype=torch.uint8)
target:  tensor([[0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 1., 0.],
        [1., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0

ValueError: Found input variables with inconsistent numbers of samples: [32, 18]